导入需要使用到的包

In [4]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 使用GPU设备
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

配置超参数

In [5]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

创建数据集

In [6]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

创建一个卷积网络。需要使用到`Conv2d`,`BatchNorm2d`,`ReLU`,`MaxPool2d`层：
 - Conv2d: 二维卷积，Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
 - BatchNorm2d :批正则
 - MaxPool2d：2D池化

In [8]:
class ConvNet(nn.Module):
    def __init__(self,output_size=10):
        super(ConvNet,self).__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer2=nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        ## padding=2表示same padding,但是两次Pooling，所以特征图大小为28/2/2=7。
        self.fc=nn.Linear(7*7*32,output_size)
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.reshape(out.size(0),-1)
        return self.fc(out)

构建模型并训练：

In [11]:
model=ConvNet(num_classes).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2332
Epoch [1/5], Step [200/600], Loss: 0.0489
Epoch [1/5], Step [300/600], Loss: 0.1306
Epoch [1/5], Step [400/600], Loss: 0.0822
Epoch [1/5], Step [500/600], Loss: 0.0479
Epoch [1/5], Step [600/600], Loss: 0.0220
Epoch [2/5], Step [100/600], Loss: 0.0707
Epoch [2/5], Step [200/600], Loss: 0.0397
Epoch [2/5], Step [300/600], Loss: 0.0919
Epoch [2/5], Step [400/600], Loss: 0.0293
Epoch [2/5], Step [500/600], Loss: 0.0366
Epoch [2/5], Step [600/600], Loss: 0.0599
Epoch [3/5], Step [100/600], Loss: 0.0413
Epoch [3/5], Step [200/600], Loss: 0.0160
Epoch [3/5], Step [300/600], Loss: 0.0082
Epoch [3/5], Step [400/600], Loss: 0.0267
Epoch [3/5], Step [500/600], Loss: 0.1212
Epoch [3/5], Step [600/600], Loss: 0.0809
Epoch [4/5], Step [100/600], Loss: 0.0452
Epoch [4/5], Step [200/600], Loss: 0.0227
Epoch [4/5], Step [300/600], Loss: 0.0842
Epoch [4/5], Step [400/600], Loss: 0.0389
Epoch [4/5], Step [500/600], Loss: 0.0217
Epoch [4/5], Step [600/600], Loss:

测试模型：
当模型中有使用到BatchNorm或者Dropout的时候，就需要在测试之前声明是测试模式：
~~~
model.eval()
~~~
这是由于训练时通常采用mini-batch，所以每一批中的mean和std大致是相同的；而测试阶段往往是单个图像的输入，不存在mini-batch的概念。所以将model改为eval模式后，BN的参数固定，并采用之前训练好的全局的mean和std。

In [12]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99.15 %


模型表现由前面的98%提升到了99%了。保存收工。

In [13]:
torch.save(model.state_dict(), 'model.ckpt')